# Model-Observation Comparison with Monet Stats

This notebook demonstrates comprehensive model-observation comparison workflows using Monet Stats. We'll explore various atmospheric variables and evaluation techniques.

In [ ]:
# Import required libraries
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

# For xarray support
import monet_stats as ms

# Set up plotting style
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

## Load Multiple Example Datasets

We'll use temperature, precipitation, and wind datasets for comprehensive comparison.

In [ ]:
# Load the example datasets
temp_df = pd.read_csv('../data/temperature_obs_mod.csv')
precip_df = pd.read_csv('../data/precipitation_obs_mod.csv')
wind_df = pd.read_csv('../data/wind_obs_mod.csv')

print("Temperature dataset shape:", temp_df.shape)
print("Precipitation dataset shape:", precip_df.shape)
print("Wind dataset shape:", wind_df.shape)

# Display basic info for each dataset
print("\nTemperature dataset:")
print(temp_df[['observed_temp', 'modeled_temp']].describe())

print("\nPrecipitation dataset:")
print(precip_df[['observed_precip', 'modeled_precip']].describe())

print("\nWind dataset:")
print(wind_df[['observed_wind_speed', 'modeled_wind_speed']].describe())

## Temperature Model-Observation Comparison

Analyze temperature forecast performance in detail.

In [ ]:
# Extract temperature data
obs_temp = temp_df['observed_temp'].values
mod_temp = temp_df['modeled_temp'].values

# Calculate comprehensive temperature metrics
temp_metrics = {
    'MAE': ms.MAE(obs_temp, mod_temp),
    'RMSE': ms.RMSE(obs_temp, mod_temp),
    'MBE': ms.MBE(obs_temp, mod_temp),
    'Correlation': ms.pearson_correlation(obs_temp, mod_temp),
    'R2': ms.R2(obs_temp, mod_temp),
    'NSE': ms.NSE(obs_temp, mod_temp),
    'mNSE': ms.mNSE(obs_temp, mod_temp),
    'rNSE': ms.rNSE(obs_temp, mod_temp)
}

print("Temperature Model Performance Metrics:")
for metric, value in temp_metrics.items():
    print(f"{metric}: {value:.4f}")

# Calculate bias by temperature range
temp_ranges = [
    ("Very Cold (< 0°C)", obs_temp < 0),
    ("Cold (0-10°C)", (obs_temp >= 0) & (obs_temp < 10)),
    ("Mild (10-20°C)", (obs_temp >= 10) & (obs_temp < 20)),
    ("Warm (20-30°C)", (obs_temp >= 20) & (obs_temp < 30)),
    ("Hot (>= 30°C)", obs_temp >= 30)
]

print("\nTemperature Range Performance:")
for range_name, mask in temp_ranges:
    if np.sum(mask) > 0:
        obs_range = obs_temp[mask]
        mod_range = mod_temp[mask]
        mae = ms.MAE(obs_range, mod_range)
        rmse = ms.RMSE(obs_range, mod_range)
        mbe = ms.MBE(obs_range, mod_range)
        corr = ms.pearson_correlation(obs_range, mod_range)
        print(f"{range_name}: MAE={mae:.3f}, RMSE={rmse:.3f}, MBE={mbe:.3f}, Corr={corr:.3f}, Count={np.sum(mask)}")

## Precipitation Model-Observation Comparison

Analyze precipitation forecast performance using both continuous and categorical metrics.

In [ ]:
# Extract precipitation data
obs_precip = precip_df['observed_precip'].values
mod_precip = precip_df['modeled_precip'].values
obs_binary = precip_df['obs_binary_precip'].values
mod_binary = precip_df['mod_binary_precip'].values

# Calculate precipitation metrics
precip_metrics = {
    'MAE': ms.MAE(obs_precip, mod_precip),
    'RMSE': ms.RMSE(obs_precip, mod_precip),
    'MBE': ms.MBE(obs_precip, mod_precip),
    'Correlation': ms.pearson_correlation(obs_precip, mod_precip),
    'R2': ms.R2(obs_precip, mod_precip)
}

print("Precipitation Model Performance Metrics (Continuous):")
for metric, value in precip_metrics.items():
    print(f"{metric}: {value:.4f}")

# Calculate contingency table metrics
print("\nBinary Precipitation Metrics:")
print(f"Accuracy: {ms.accuracy(obs_binary, mod_binary):.4f}")
print(f"POD (Probability of Detection): {ms.POD(obs_binary, mod_binary):.4f}")
print(f"FAR (False Alarm Ratio): {ms.FAR(obs_binary, mod_binary):.4f}")
print(f"CSI (Critical Success Index): {ms.CSI(obs_binary, mod_binary):.4f}")
print(f"Heidke Skill Score: {ms.HSS(obs_binary, mod_binary):.4f}")
print(f"Brier Score: {ms.BS(obs_binary, mod_precip/np.max(mod_precip)):.4f}")

# Calculate precipitation-specific metrics
print("\nRainfall Intensity Metrics:")
print(f"Mean Observed Rainfall: {np.mean(obs_precip):.4f} mm/day")
print(f"Mean Modeled Rainfall: {np.mean(mod_precip):.4f} mm/day")
print(f"Bias Ratio: {np.mean(mod_precip) / np.mean(obs_precip):.4f}")

# Calculate metrics for wet days only (>0.1mm)
wet_days = obs_precip > 0.1
if np.sum(wet_days) > 0:
    obs_wet = obs_precip[wet_days]
    mod_wet = mod_precip[wet_days]
    print(f"Wet Day RMSE: {ms.RMSE(obs_wet, mod_wet):.4f}")
    print(f"Wet Day Correlation: {ms.pearson_correlation(obs_wet, mod_wet):.4f}")

## Wind Model-Observation Comparison

Analyze wind speed and direction forecast performance.

In [ ]:
# Extract wind data
obs_wind_speed = wind_df['observed_wind_speed'].values
mod_wind_speed = wind_df['modeled_wind_speed'].values
obs_wind_dir = wind_df['observed_wind_dir'].values
mod_wind_dir = wind_df['modeled_wind_dir'].values

# Calculate wind speed metrics
wind_speed_metrics = {
    'MAE': ms.MAE(obs_wind_speed, mod_wind_speed),
    'RMSE': ms.RMSE(obs_wind_speed, mod_wind_speed),
    'MBE': ms.MBE(obs_wind_speed, mod_wind_speed),
    'Correlation': ms.pearson_correlation(obs_wind_speed, mod_wind_speed),
    'R2': ms.R2(obs_wind_speed, mod_wind_speed)
}

print("Wind Speed Model Performance Metrics:")
for metric, value in wind_speed_metrics.items():
    print(f"{metric}: {value:.4f}")

# Calculate wind direction metrics (circular statistics)
print("\nWind Direction Metrics:")

# Calculate mean absolute error for wind direction (considering circular nature)
def circular_mae(obs_dir, mod_dir):
    """Calculate circular mean absolute error for wind direction"""
    # Calculate the minimum angular difference
    diff = np.abs(obs_dir - mod_dir)
    diff = np.minimum(diff, 360 - diff)
    return np.mean(diff)

dir_mae = circular_mae(obs_wind_dir, mod_wind_dir)
print(f"Circular Mean Absolute Error (Direction): {dir_mae:.4f} degrees")

# Calculate correlation for wind speed
print(f"Wind Speed Correlation: {ms.pearson_correlation(obs_wind_speed, mod_wind_speed):.4f}")

# Calculate metrics for different wind speed ranges
wind_ranges = [
    ("Light (0-5 m/s)", obs_wind_speed < 5),
    ("Moderate (5-10 m/s)", (obs_wind_speed >= 5) & (obs_wind_speed < 10)),
    ("Strong (10-15 m/s)", (obs_wind_speed >= 10) & (obs_wind_speed < 15)),
    ("Very Strong (>= 15 m/s)", obs_wind_speed >= 15)
]

print("\nWind Speed Range Performance:")
for range_name, mask in wind_ranges:
    if np.sum(mask) > 0:
        obs_range = obs_wind_speed[mask]
        mod_range = mod_wind_speed[mask]
        mae = ms.MAE(obs_range, mod_range)
        rmse = ms.RMSE(obs_range, mod_range)
        mbe = ms.MBE(obs_range, mod_range)
        corr = ms.pearson_correlation(obs_range, mod_range)
        print(f"{range_name}: MAE={mae:.3f}, RMSE={rmse:.3f}, MBE={mbe:.3f}, Corr={corr:.3f}, Count={np.sum(mask)}")

## Comprehensive Model Evaluation Dashboard

Create a comprehensive dashboard comparing all variables.

In [ ]:
# Create a comprehensive dashboard
fig, axes = plt.subplots(3, 3, figsize=(18, 15))

# Temperature scatter plot
axes[0, 0].scatter(obs_temp[:1000], mod_temp[:1000], alpha=0.5, s=10)
axes[0, 0].plot([obs_temp.min(), obs_temp.max()], [obs_temp.min(), obs_temp.max()], 'r--', lw=2)
axes[0, 0].set_xlabel('Observed Temperature (°C)')
axes[0, 0].set_ylabel('Modeled Temperature (°C)')
axes[0, 0].set_title(f'Temperature Scatter (R² = {ms.R2(obs_temp, mod_temp):.3f})')
axes[0, 0].grid(True, alpha=0.3)

# Precipitation scatter plot
axes[0, 1].scatter(obs_precip[:1000], mod_precip[:1000], alpha=0.5, s=10)
axes[0, 1].plot([obs_precip.min(), obs_precip.max()], [obs_precip.min(), obs_precip.max()], 'r--', lw=2)
axes[0, 1].set_xlabel('Observed Precipitation (mm)')
axes[0, 1].set_ylabel('Modeled Precipitation (mm)')
axes[0, 1].set_title(f'Precipitation Scatter (R² = {ms.R2(obs_precip, mod_precip):.3f})')
axes[0, 1].set_xlim(0, np.percentile(obs_precip, 95))
axes[0, 1].set_ylim(0, np.percentile(mod_precip, 95))
axes[0, 1].grid(True, alpha=0.3)

# Wind speed scatter plot
axes[0, 2].scatter(obs_wind_speed[:1000], mod_wind_speed[:1000], alpha=0.5, s=10)
axes[0, 2].plot([obs_wind_speed.min(), obs_wind_speed.max()], [obs_wind_speed.min(), obs_wind_speed.max()], 'r--', lw=2)
axes[0, 2].set_xlabel('Observed Wind Speed (m/s)')
axes[0, 2].set_ylabel('Modeled Wind Speed (m/s)')
axes[0, 2].set_title(f'Wind Speed Scatter (R² = {ms.R2(obs_wind_speed, mod_wind_speed):.3f})')
axes[0, 2].grid(True, alpha=0.3)

# Temperature time series for first station
first_temp_station = temp_df[temp_df['station_id'] == temp_df['station_id'].iloc[0]]
axes[1, 0].plot(first_temp_station['date'][:365], first_temp_station['observed_temp'][:365], label='Observed', alpha=0.7)
axes[1, 0].plot(first_temp_station['date'][:365], first_temp_station['modeled_temp'][:365], label='Modeled', alpha=0.7)
axes[1, 0].set_xlabel('Date')
axes[1, 0].set_ylabel('Temperature (°C)')
axes[1, 0].set_title('Temperature Time Series Comparison')
axes[1, 0].legend()
axes[1, 0].tick_params(axis='x', rotation=45)

# Precipitation time series for first station
first_precip_station = precip_df[precip_df['station_id'] == precip_df['station_id'].iloc[0]]
axes[1, 1].plot(first_precip_station['date'][:365], first_precip_station['observed_precip'][:365], label='Observed', alpha=0.7)
axes[1, 1].plot(first_precip_station['date'][:365], first_precip_station['modeled_precip'][:365], label='Modeled', alpha=0.7)
axes[1, 1].set_xlabel('Date')
axes[1, 1].set_ylabel('Precipitation (mm)')
axes[1, 1].set_title('Precipitation Time Series Comparison')
axes[1, 1].legend()
axes[1, 1].tick_params(axis='x', rotation=45)

# Wind speed time series for first station
first_wind_station = wind_df[wind_df['station_id'] == wind_df['station_id'].iloc[0]]
axes[1, 2].plot(first_wind_station['date'][:168], first_wind_station['observed_wind_speed'][:168], label='Observed', alpha=0.7)  # 1 week of hourly data
axes[1, 2].plot(first_wind_station['date'][:168], first_wind_station['modeled_wind_speed'][:168], label='Modeled', alpha=0.7)
axes[1, 2].set_xlabel('Date')
axes[1, 2].set_ylabel('Wind Speed (m/s)')
axes[1, 2].set_title('Wind Speed Time Series Comparison (1 Week)')
axes[1, 2].legend()
axes[1, 2].tick_params(axis='x', rotation=45)

# Temperature error histogram
temp_errors = mod_temp - obs_temp
axes[2, 0].hist(temp_errors, bins=50, edgecolor='black', alpha=0.7)
axes[2, 0].set_xlabel('Temperature Error (°C)')
axes[2, 0].set_ylabel('Frequency')
axes[2, 0].set_title(f'Temperature Error Distribution (Mean: {np.mean(temp_errors):.3f})')
axes[2, 0].grid(True, alpha=0.3)

# Precipitation error histogram
precip_errors = mod_precip - obs_precip
axes[2, 1].hist(precip_errors, bins=50, edgecolor='black', alpha=0.7)
axes[2, 1].set_xlabel('Precipitation Error (mm)')
axes[2, 1].set_ylabel('Frequency')
axes[2, 1].set_title(f'Precipitation Error Distribution (Mean: {np.mean(precip_errors):.3f})')
axes[2, 1].grid(True, alpha=0.3)

# Wind speed error histogram
wind_errors = mod_wind_speed - obs_wind_speed
axes[2, 2].hist(wind_errors, bins=50, edgecolor='black', alpha=0.7)
axes[2, 2].set_xlabel('Wind Speed Error (m/s)')
axes[2, 2].set_ylabel('Frequency')
axes[2, 2].set_title(f'Wind Speed Error Distribution (Mean: {np.mean(wind_errors):.3f})')
axes[2, 2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## Performance by Season

Analyze model performance across different seasons.

In [ ]:
# Add seasonal information to temperature data
temp_df['date'] = pd.to_datetime(temp_df['date'])
temp_df['month'] = temp_df['date'].dt.month
temp_df['season'] = temp_df['month'].map({12: 'Winter', 1: 'Winter', 2: 'Winter',
                                         3: 'Spring', 4: 'Spring', 5: 'Spring',
                                         6: 'Summer', 7: 'Summer', 8: 'Summer',
                                         9: 'Fall', 10: 'Fall', 11: 'Fall'})

# Calculate metrics by season
seasons = ['Winter', 'Spring', 'Summer', 'Fall']
seasonal_metrics = []

for season in seasons:
    season_data = temp_df[temp_df['season'] == season]
    obs_season = season_data['observed_temp'].values
    mod_season = season_data['modeled_temp'].values

    if len(obs_season) > 0:
        seasonal_metrics.append({
            'season': season,
            'MAE': ms.MAE(obs_season, mod_season),
            'RMSE': ms.RMSE(obs_season, mod_season),
            'MBE': ms.MBE(obs_season, mod_season),
            'Correlation': ms.pearson_correlation(obs_season, mod_season),
            'R2': ms.R2(obs_season, mod_season),
            'Count': len(obs_season)
        })

# Convert to DataFrame and display
seasonal_df = pd.DataFrame(seasonal_metrics)
print("Seasonal Performance Metrics (Temperature):")
print(seasonal_df)

# Visualize seasonal performance
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

axes[0, 0].bar(seasonal_df['season'], seasonal_df['RMSE'])
axes[0, 0].set_xlabel('Season')
axes[0, 0].set_ylabel('RMSE')
axes[0, 0].set_title('RMSE by Season')
axes[0, 0].grid(True, alpha=0.3)

axes[0, 1].bar(seasonal_df['season'], seasonal_df['MAE'])
axes[0, 1].set_xlabel('Season')
axes[0, 1].set_ylabel('MAE')
axes[0, 1].set_title('MAE by Season')
axes[0, 1].grid(True, alpha=0.3)

axes[1, 0].bar(seasonal_df['season'], seasonal_df['Correlation'])
axes[1, 0].set_xlabel('Season')
axes[1, 0].set_ylabel('Correlation')
axes[1, 0].set_title('Correlation by Season')
axes[1, 0].grid(True, alpha=0.3)

axes[1, 1].bar(seasonal_df['season'], seasonal_df['MBE'])
axes[1, 1].set_xlabel('Season')
axes[1, 1].set_ylabel('MBE')
axes[1, 1].set_title('MBE by Season')
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## Summary

This notebook demonstrated comprehensive model-observation comparison workflows:

1. **Temperature Analysis**: Continuous metrics, performance by temperature range
2. **Precipitation Analysis**: Both continuous and categorical metrics, binary event detection
3. **Wind Analysis**: Speed and direction metrics, performance by wind speed range
4. **Comprehensive Dashboard**: Multi-variable visualization and comparison
5. **Seasonal Analysis**: Performance variation across seasons

These workflows provide a complete framework for evaluating atmospheric model performance across multiple variables and conditions.